## Window Functions 
- 테이블에서 행집합을 대상으로 하는 함수
- 집합 단위로 계산하기 때문에, 집계 함수와 비슷
- 단, 집계 함수는 한 행으로 결괏값을 보여주는 반면, 윈도우 암수는 각 행마다 처리 결과를 출력함
- 윈도우 함수를 사용하려면 집약함수 뒤에 OVER를 붙이고 윈도 함수를 지정합니다. 

### ROW Number()
- 각 행에 숫자를 1, 2, 3, ..., N 형태로 추가하도록 합니다.
- Row_N 기준으로 오름차순으로 진행합니다.  

```SQL
SELECT
  ROW_NUMBER() OVER() AS Row_N 
  , *
FROM Summer_Medals
ORDER BY Row_N ASC;
 row_n | year |  city  |  sport   | discipline |      athlete      | country | gender |           event            | medal  
-------+------+--------+----------+------------+-------------------+---------+--------+----------------------------+--------
     1 | 1896 | Athens | Aquatics | Swimming   | HAJOS Alfred      | HUN     | Men    | 100M Freestyle             | Gold
     2 | 1896 | Athens | Aquatics | Swimming   | HERSCHMANN Otto   | AUT     | Men    | 100M Freestyle             | Silver
     3 | 1896 | Athens | Aquatics | Swimming   | DRIVAS Dimitrios  | GRE     | Men    | 100M Freestyle For Sailors | Bronze
     4 | 1896 | Athens | Aquatics | Swimming   | MALOKINIS Ioannis | GRE     | Men    | 100M Freestyle For Sailors | Gold
     5 | 1896 | Athens | Aquatics | Swimming   | CHASAPIS Spiridon | GRE     | Men    | 100M Freestyle For Sailors | Silver
(5 rows)

```



- 이번에는 올림픽 년도를 오름차순 순번대로 작성을 하도록 합니다. 
- 이 때, 중요한 건 서브쿼리로 연도만을 추출한 뒤, 윈도우 함수를 이용한 것입니다. 

```SQL
SELECT 
    Year, 
    ROW_NUMBER() OVER() AS Row_N
FROM (
    SELECT DISTINCT Year
    FROM summer_medals
    ORDER BY Year ASC
) AS Years
ORDER BY Year ASC;
 year | row_n 
------+-------
 1896 |     1
 1900 |     2
 1904 |     3
 1908 |     4
 1912 |     5
(5 rows)

```



## ORDER BY
- 하계 올림픽이 열린 각 연도에 번호를 할당합니다. 
- 가장 최근 연도를 가진 행이 더 낮은 행 수를 갖도록 합니다.

```SQL
SELECT
  Year,
  -- Assign the lowest numbers to the most recent years
  ROW_NUMBER() OVER (ORDER BY Year DESC) AS Row_N
FROM (
  SELECT DISTINCT Year
  FROM Summer_Medals
) AS Years
ORDER BY Year;
```

### ORDER BY
- 각 운동선수들이 획득한 메달 갯수를 내림차순으로 정렬하도록 합니다. 

```SQL
SELECT
  Athlete,
  COUNT(*) AS Medals
FROM Summer_Medals
GROUP BY Athlete
ORDER BY Medals DESC
LIMIT 5;
       athlete       | medals 
---------------------+--------
 PHELPS Michael      |     22
 LATYNINA Larisa     |     18
 ANDRIANOV Nikolay   |     15
 MANGIAROTTI Edoardo |     13
 SHAKHLIN Boris      |     13
(5 rows)

```
- 이전 쿼리에서, 각 선수들의 랭킹을 추가하도록 합니다. 

```SQL
WITH Athlete_Medals AS (
  SELECT
    -- Count the number of medals each athlete has earned
    Athlete,
    COUNT(*) AS Medals
  FROM Summer_Medals
  GROUP BY Athlete)

SELECT
  -- Number each athlete by how many medals they've earned
  Medals, 
  Athlete,
  ROW_NUMBER() OVER (ORDER BY Medals DESC) AS Row_N
FROM Athlete_Medals
ORDER BY Medals DESC
LIMIT 5;
 medals |       athlete       | row_n 
--------+---------------------+-------
     22 | PHELPS Michael      |     1
     18 | LATYNINA Larisa     |     2
     15 | ANDRIANOV Nikolay   |     3
     13 | MANGIAROTTI Edoardo |     4
     13 | ONO Takashi         |     5
(5 rows)

```

- 남자 69KG 역도 경기에서 매년 금메달리스트 조회하도록 합니다. 

```SQL
SELECT
    -- Return each year's champions' countries
    Year,
    Country AS champion
  FROM Summer_Medals
  WHERE
    Discipline = 'Weightlifting' AND
    Event = '69KG' AND
    Gender = 'Men' AND
    Medal = 'Gold';
 year | champion 
------+----------
 2000 | BUL   
 2004 | CHN   
 2008 | CHN   
 2012 | CHN   
(4 rows)
```

- 기존 쿼리에서 매년 전년도 챔피언도 같이 조회하도록 합니다. 

```SQL
WITH Weightlifting_Gold AS (
  SELECT
    -- Return each year's champions' countries
    Year,
    Country AS champion
  FROM Summer_Medals
  WHERE
    Discipline = 'Weightlifting' AND
    Event = '69KG' AND
    Gender = 'Men' AND
    Medal = 'Gold')

SELECT
  Year, Champion,
  -- Fetch the previous year's champion
  LAG(Champion) OVER
    (ORDER BY Year ASC) AS Last_Champion
FROM Weightlifting_Gold
ORDER BY Year ASC;
 year | champion | last_champion 
------+----------+---------------
 2000 | BUL      | 
 2004 | CHN      | BUL   
 2008 | CHN      | CHN   
 2012 | CHN      | CHN   
(4 rows)

```


## LAG() 함수의 활용
- Lag() 함수는 이전 행의 컬럼 값과 비교를 하거나 또는 값을 가져올 때 사용합니다. 
- 이 때 사용하는 문법이 LAG() OVER() 구문입니다. 

- 우선, 200M 달리기 금메달을 확인해보도록 한다. 

```SQL
SELECT 
	Year, 
	Athlete, 
	Country AS Champion
FROM summer_medals
WHERE 
	Discipline = 'Athletics' AND 
	Event = '200M' AND 
	Gender = 'Men' AND 
	Medal = 'Gold';

 year |           athlete            | champion 
------+------------------------------+----------
 1900 | TEWKSBURY Walter B. John     | USA
 1904 | HAHN Archie                  | USA
 1908 | KERR Robert                  | CAN
 1912 | CRAIG Ralph                  | USA
 1920 | WOODRING Allen               | USA
 1924 | SCHOLZ Jackson               | USA
 1928 | WILLIAMS Percy               | CAN
 1932 | TOLAN Eddie                  | USA
 1936 | OWENS Jesse                  | USA
 1948 | PATTON Melvin Emery          | USA
 1952 | STANFIELD Andrew William     | USA
 1956 | MORROW Robert Joseph         | USA
 1960 | BERRUTI Livio                | ITA
 1964 | CARR Henry                   | USA
 1968 | SMITH Thomas C.              | USA
 1972 | BORZOV Valery                | URS
 1976 | QUARRIE Donald               | JAM
 1980 | MENNEA Pietro                | ITA
 1984 | LEWIS Carl                   | USA
 1988 | DELOACH Joseph Nathaniel Jr. | USA
 1992 | MARSH Michael                | USA
 1996 | JOHNSON Michael              | USA
 2000 | KENTERIS Konstantinos        | GRE
 2004 | CRAWFORD Shawn               | USA
 2008 | BOLT Usain                   | JAM
 2012 | BOLT Usain                   | JAM
 
(26 rows)

```

- 그러나, 여기에서 한걸음 더 나아가, 2연패를 달성한 선수가 있는지 확인을 해보도록 한다. 

```SQL
WITH Gold AS (
SELECT 
	Year, 
	Athlete AS Champion
FROM summer_medals
WHERE 
	Discipline = 'Athletics' AND 
	Event = '200M' AND 
	Gender = 'Men' AND 
	Medal = 'Gold')

SELECT 
	Year, Champion,
	LAG(Champion) OVER (ORDER BY YEAR ASC) AS Last_Champion
FROM Gold
ORDER BY Year ASC;
 year |           champion           |        last_champion         
------+------------------------------+------------------------------
 1900 | TEWKSBURY Walter B. John     | 
 1904 | HAHN Archie                  | TEWKSBURY Walter B. John
 1908 | KERR Robert                  | HAHN Archie
 1912 | CRAIG Ralph                  | KERR Robert
 1920 | WOODRING Allen               | CRAIG Ralph
 1924 | SCHOLZ Jackson               | WOODRING Allen
 1928 | WILLIAMS Percy               | SCHOLZ Jackson
 1932 | TOLAN Eddie                  | WILLIAMS Percy
 1936 | OWENS Jesse                  | TOLAN Eddie
 1948 | PATTON Melvin Emery          | OWENS Jesse
 1952 | STANFIELD Andrew William     | PATTON Melvin Emery
 1956 | MORROW Robert Joseph         | STANFIELD Andrew William
 1960 | BERRUTI Livio                | MORROW Robert Joseph
 1964 | CARR Henry                   | BERRUTI Livio
 1968 | SMITH Thomas C.              | CARR Henry
 1972 | BORZOV Valery                | SMITH Thomas C.
 1976 | QUARRIE Donald               | BORZOV Valery
 1980 | MENNEA Pietro                | QUARRIE Donald
 1984 | LEWIS Carl                   | MENNEA Pietro
 1988 | DELOACH Joseph Nathaniel Jr. | LEWIS Carl
 1992 | MARSH Michael                | DELOACH Joseph Nathaniel Jr.
 1996 | JOHNSON Michael              | MARSH Michael
 2000 | KENTERIS Konstantinos        | JOHNSON Michael
 2004 | CRAWFORD Shawn               | KENTERIS Konstantinos
 2008 | BOLT Usain                   | CRAWFORD Shawn
 2012 | BOLT Usain                   | BOLT Usain
(26 rows)

```

## Partition by
- Partition BY는 열의 고유 값을 기준으로 테이블을 파티션으로 분할함. 
    - 결과가 한 열로 롤업되지 않음 
- 창 기능에 따라 별도로 작동됨
    - ROW_NUMBER가 각 파티션에 대해 재설정됩니다.
    - LAG는 이전 행이 동일한 파티션에 있는 경우에만 이전 값을 가져옵니다.

### Example
- 아래 쿼리 결괏값에 대한 문제점을 확인합니다. 

```SQL
WITH Discus_Gold_Medal AS (
	SELECT 
		Year, Event, Country AS Champion
	FROM summer_medals
	WHERE 
		Year IN (2004, 2008, 2012)
		AND Gender = 'Men' AND Medal = 'Gold'
	    AND Event IN ('Discus Throw', 'Triple Jump')
		AND Gender = 'Men')
		
SELECT 
	YEAR, Event, Champion, 
	LAG(Champion) OVER
	(ORDER BY Event ASC, Year ASC) AS Last_Champion
FROM Discus_Gold_Medal
ORDER BY Event ASC, Year ASC;
```
```
 year |    event     | champion | last_champion 
------+--------------+----------+---------------
 2004 | Discus Throw | LTU      | 
 2008 | Discus Throw | EST      | LTU   
 2012 | Discus Throw | GER      | EST   
 2004 | Triple Jump  | SWE      | GER   
 2008 | Triple Jump  | POR      | SWE   
 2012 | Triple Jump  | USA      | POR   
(6 rows)
```

- Triple Jump의 GER가 나타나는 것은 어색하다. 
- 이번에는 Partition By 문법을 적용합니다. 

```SQL
WITH Discus_Gold_Medal AS (
	SELECT 
		Year, Event, Country AS Champion
	FROM summer_medals
	WHERE 
		Year IN (2004, 2008, 2012)
		AND Gender = 'Men' AND Medal = 'Gold'
	    AND Event IN ('Discus Throw', 'Triple Jump')
		AND Gender = 'Men')
		
SELECT 
        YEAR, Event, Champion, 
        LAG(Champion) OVER
        (PARTITION BY Event ORDER BY Event ASC, Year ASC) AS Last_Champion
FROM Discus_Gold_Medal
ORDER BY Event ASC, Year ASC;
```
```
 year |    event     | champion | last_champion 
------+--------------+----------+---------------
 2004 | Discus Throw | LTU      | 
 2008 | Discus Throw | EST      | LTU   
 2012 | Discus Throw | GER      | EST   
 2004 | Triple Jump  | SWE      | 
 2008 | Triple Jump  | POR      | SWE   
 2012 | Triple Jump  | USA      | POR   
(6 rows)
```


- 성별에 따라 각 이벤트의 이전 챔피언을 반환합니다.

```SQL
WITH Tennis_Gold AS (
  SELECT DISTINCT
    Gender, Year, Country
  FROM Summer_Medals
  WHERE
    Year >= 2000 AND
    Event = 'Javelin Throw' AND
    Medal = 'Gold')

SELECT
  Gender, Year,
  Country AS Champion,
  -- Fetch the previous year's champion by gender
  LAG(Country) OVER (PARTITION BY Gender
                         ORDER BY Year ASC) AS Last_Champion
FROM Tennis_Gold
ORDER BY Gender ASC, Year ASC LIMIT 10;
```
```
 gender | year | champion | last_champion 
--------+------+----------+---------------
 Men    | 2000 | CZE      | 
 Men    | 2004 | NOR      | CZE   
 Men    | 2008 | NOR      | NOR   
 Men    | 2012 | TTO      | NOR   
 Women  | 2000 | NOR      | 
 Women  | 2004 | CUB      | NOR   
 Women  | 2008 | CZE      | CUB   
 Women  | 2012 | CZE      | CZE   
(8 rows)
```

- 이번에는 경기종목을 2개로 추가하도록 합니다. ('200M', '10000M')
    + 이 때에는 IN() 함수를 사용합니다. 
    
```SQL
WITH Athletics_Gold AS (
  SELECT DISTINCT
    Gender, Year, Event, Country
  FROM Summer_Medals
  WHERE
    Year >= 2000 AND
    Discipline = 'Athletics' AND
    Event IN ('100M', '10000M') AND
    Medal = 'Gold')

SELECT
  Gender, Year, Event,
  Country AS Champion,
  -- Fetch the previous year's champion by gender and event
  LAG(Country) OVER (PARTITION BY Gender, Event
                         ORDER BY Year ASC) AS Last_Champion
FROM Athletics_Gold
ORDER BY Event ASC, Gender ASC, Year ASC 
LIMIT 10;
```
```
 gender | year | event  | champion | last_champion 
--------+------+--------+----------+---------------
 Men    | 2000 | 10000M | ETH      | 
 Men    | 2004 | 10000M | ETH      | ETH   
 Men    | 2008 | 10000M | ETH      | ETH   
 Men    | 2012 | 10000M | GBR      | ETH   
 Women  | 2000 | 10000M | ETH      | 
 Women  | 2004 | 10000M | CHN      | ETH   
 Women  | 2008 | 10000M | ETH      | CHN   
 Women  | 2012 | 10000M | ETH      | ETH   
 Men    | 2000 | 100M   | USA      | 
 Men    | 2004 | 100M   | USA      | USA   
(10 rows)

```

## Relative vs Absolute
- Relative
    + LAG(column, n) LAG()는 현재 행 앞의 행에 있는 열의 값을 반환합니다. 
    + Lead(column, n) 현재 행 뒤의 행에 있는 열의 값을 반환합니다.
- Absolute
    + FIRST_VALUE(column) 테이블 또는 파티션의 첫번째 값을 반환합니다.
    + LAST_VALUE(column) 테이블 또는 파티션의 마지막 값을 반환합니다.

### Question 1. 
- 각 연도마다 현재 금메달리스트와 3개 대회 이후의 메달리스트를 같이 조회합니다. 

```SQL
WITH Discus_Medalists AS (
  SELECT DISTINCT
    Year,
    Athlete
  FROM Summer_Medals
  WHERE Medal = 'Gold'
    AND Event = 'Discus Throw'
    AND Gender = 'Women'
    AND Year >= 1992)

SELECT
  -- For each year, fetch the current and future medalists
  Year,
  Athlete,
  LEAD(Athlete, 3) OVER (ORDER BY Year ASC) AS Future_Champion
FROM Discus_Medalists
ORDER BY Year ASC;
```
```
 year |         athlete         |     future_champion     
------+-------------------------+-------------------------
 1992 | MARTEN Maritza          | SADOVA Natalya
 1996 | WYLUDDA Ilke            | BROWN TRAFTON Stephanie
 2000 | ZVEREVA Ellina          | PERKOVIC Sandra
 2004 | SADOVA Natalya          | 
 2008 | BROWN TRAFTON Stephanie | 
 2012 | PERKOVIC Sandra         | 
(6 rows)
```

### Question 2. 
- 모든 선수와 첫 번째 선수를 알파벳 순서대로 조회합니다.

```SQL
WITH All_Male_Medalists AS (
  SELECT DISTINCT
    Athlete
  FROM Summer_Medals
  WHERE Medal = 'Gold'
    AND Gender = 'Men')

SELECT
  -- Fetch all athletes and the first athlete alphabetically
  Athlete,
  FIRST_VALUE(Athlete) OVER (
    ORDER BY Athlete ASC
  ) AS First_Athlete
FROM All_Male_Medalists 
LIMIT 5;
```
```
         athlete         | first_athlete 
-------------------------+---------------
 AABYE Edgar             | AABYE Edgar
 AALTONEN Paavo Johannes | AABYE Edgar
 AAS Thomas Valentin     | AABYE Edgar
 ABALMASAU Aliaksei      | AABYE Edgar
 ABALO Luc               | AABYE Edgar
(5 rows)

```

### Question 3. 
- 각 올림픽 경기가 열렸던 해외 도시를 조회합니다. 
- 본 데이터에서 올림픽 경기가 열렸던 마지막 도시를 조회합니다. 

```SQL
WITH Hosts AS (
  SELECT DISTINCT Year, City
    FROM Summer_Medals)

SELECT
  Year,
  City,
  -- Get the last city in which the Olympic games were held
  LAST_VALUE(City) OVER (
   ORDER BY Year ASC
   RANGE BETWEEN
     UNBOUNDED PRECEDING AND
     UNBOUNDED FOLLOWING
  ) AS Last_City
FROM Hosts
ORDER BY Year ASC;
```
```
 year |         city          | last_city 
------+-----------------------+-----------
 1896 | Athens                | London
 1900 | Paris                 | London
 1904 | St Louis              | London
 1908 | London                | London
 1912 | Stockholm             | London
 1920 | Antwerp               | London
 1924 | Paris                 | London
 1928 | Amsterdam             | London
 1932 | Los Angeles           | London
 1936 | Berlin                | London
 1948 | London                | London
 1952 | Helsinki              | London
 1956 | Melbourne / Stockholm | London
 1960 | Rome                  | London
 1964 | Tokyo                 | London
 1968 | Mexico                | London
 1972 | Munich                | London
 1976 | Montreal              | London
 1980 | Moscow                | London
 1984 | Los Angeles           | London
 1988 | Seoul                 | London
 1992 | Barcelona             | London
 1996 | Atlanta               | London
 2000 | Sydney                | London
 2004 | Athens                | London
 2008 | Beijing               | London
 2012 | London                | London
(27 rows)

```



## Ranking 함수 
- ROW_NUMBER()는 두 행의 값이 동일한 경우에도 항상 고유한 번호를 할당합니다.
- RANK() 동일한 값을 가진 행에 같은 번호를 할당하고 이러한 경우 다음 숫자를 건너뜁니다.
- DENSE_RANK() 값이 동일한 행에 같은 숫자를 할당하지만 다음 숫자를 건너뛰지 않습니다.

### Example
- 9개 국가의 참여 횟수를 구해보도록 합니다. 

```SQL
SELECT 
	Country, COUNT(DISTINCT Year) AS Games
FROM summer_medals
Where 
	Country IN ('GBR', 'DEN', 'FRA', 
			    'ITA', 'AUT', 'BEL', 
			    'NOR', 'JPN', 'KOR')
GROUP BY Country
ORDER BY GAMES DESC;
```
```
 country | games 
---------+-------
 GBR     |    27
 DEN     |    26
 FRA     |    26
 ITA     |    25
 AUT     |    24
 BEL     |    24
 NOR     |    22
 JPN     |    20
 KOR     |    15
(9 rows)

```

- 3개의 함수를 비교합니다. 

```SQL
WITH Country_Games AS (
   SELECT 
       Country, COUNT(DISTINCT Year) AS Games
       FROM summer_medals
       Where 
          Country IN ('GBR', 'DEN', 'FRA', 
                              'ITA', 'AUT', 'BEL', 
                            'NOR', 'JPN', 'KOR')
        GROUP BY Country
        ORDER BY GAMES DESC)
SELECT 
     Country, Games, 
         ROW_NUMBER()
	OVER (ORDER BY Games DESC) AS Row_N,
         RANK()
	OVER (ORDER BY Games DESC) AS Rank_N, 
         DENSE_RANK()
	OVER (ORDER BY Games DESC) AS Dense_Rank_N
FROM Country_Games
ORDER BY Games DESC, Country ASC;
```
```
 country | games | row_n | rank_n | dense_rank_n 
---------+-------+-------+--------+--------------
 GBR     |    27 |     1 |      1 |            1
 DEN     |    26 |     2 |      2 |            2
 FRA     |    26 |     3 |      2 |            2
 ITA     |    25 |     4 |      4 |            3
 AUT     |    24 |     5 |      5 |            4
 BEL     |    24 |     6 |      5 |            4
 NOR     |    22 |     7 |      7 |            5
 JPN     |    20 |     8 |      8 |            6
 KOR     |    15 |     9 |      9 |            7
(9 rows)
```

### Question 1.
- 메달 개수에 따라 각 선수의 순위를 매깁니다. 
- 숫자가 높을수록, 랭킹이 높을수록, 값이 같을 경우 숫자가 동일하다.

```SQL
WITH Athlete_Medals AS (
  SELECT
    Athlete,
    COUNT(*) AS Medals
  FROM Summer_Medals
  GROUP BY Athlete)

SELECT
  Athlete,
  Medals,
  -- Rank athletes by the medals they've won
  RANK() OVER (ORDER BY Medals DESC) AS Rank_N
FROM Athlete_Medals
ORDER BY Medals DESC
LIMIT 10;
```
```
       athlete       | medals | rank_n 
---------------------+--------+--------
 PHELPS Michael      |     22 |      1
 LATYNINA Larisa     |     18 |      2
 ANDRIANOV Nikolay   |     15 |      3
 MANGIAROTTI Edoardo |     13 |      4
 ONO Takashi         |     13 |      4
 SHAKHLIN Boris      |     13 |      4
 THOMPSON Jenny      |     12 |      7
 NEMOV Alexei        |     12 |      7
 FISCHER Birgit      |     12 |      7
 KATO Sawao          |     12 |      7
(10 rows)

```

### Question 2. 
- 각 나라의 운동선수들을 그들이 획득한 메달의 개수로 순위를 매깁니다. 

```SQL
WITH Athlete_Medals AS (
  SELECT
    Country, Athlete, COUNT(*) AS Medals
  FROM Summer_Medals
  WHERE
    Country IN ('JPN', 'KOR')
    AND Year >= 2000
  GROUP BY Country, Athlete
  HAVING COUNT(*) > 1)

SELECT
  Country,
  -- Rank athletes in each country by the medals they've won
  Athlete,
  DENSE_RANK() OVER (PARTITION BY Country
                         ORDER BY Medals DESC) AS Rank_N
FROM Athlete_Medals
ORDER BY Country ASC, RANK_N ASC;
```
```
 country |     athlete      | rank_n 
---------+------------------+--------
 JPN     | KITAJIMA Kosuke  |      1
 JPN     | UCHIMURA Kohei   |      2
 .
 .
 JPN     | SAITO Haruka     |      5
 JPN     | UENO Masae       |      5
 KOR     | JIN Jongoh       |      1
 KOR     | PARK Taehwan     |      2
 .
 .
 KOR     | OH Seongok       |      4
 KOR     | LEE Dong Soo     |      4
 KOR     | LEE Kyungwon     |      4
(74 rows)

```